In [10]:
import numpy as np
from scipy.optimize import fmin
from astropy.stats import bootstrap
from random import shuffle
import random
import time
from astropy.stats import bootstrap

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
maze_ori = np.matrix("99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99;\
                 99, 0, 1, 0, 2, 0, 8, 0, 5, 0, 3, 0, 1, 0, 1, 0, 99;\
           99, -2, 99, 2, 99, 10, 99, 5, 99, -1, 99, 5, 99, 10, 99, -1, 99;\
           99, 0, 2, 0, 7, 0, -1, 0, 2, 0, 2, 0, 3, 0, 5, 0, 99;\
           99, 7, 99, 5, 99, 2, 99, -1, 99 2, 99, 3, 99, 8, 99, 1, 99;\
           99, 0, 3, 0, 2, 0, 6, 0, 4, 0, 2, 0, 1, 0, 8, 0, 99;\
           99, 4, 99, 6, 99, -1, 99, 2, 99, -1, 99, 8, 99, -1, 99, 8, 99;\
           99, 0, 2, 0, 5, 0, 3, 0, 4, 0, 1, 0, -1, 0, 3, 0, 99;\
           99, 2, 99, 10, 99, 4, 99, 1, 99, 9, 99, 4, 99, 20, 99, 5, 99;\
           99, 0, -2, 0, 3, 0, 4, 0, 7, 0, -1, 0, 1, 0, 3, 0, 99;\
           99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99")
maze_ori = maze_ori.tolist()

# dirs=[(0,1),(1,0),(0,-1),(-1,0)]
path=[]

def mark(maze,pos):  #给迷宫maze的位置pos标"2"表示“倒过了”
    if maze[pos[0]][pos[1]] == 0:
        maze[pos[0]][pos[1]]=0
    else:
        maze[pos[0]][pos[1]]=99
    return maze

def passable(maze,pos): #检查迷宫maze的位置pos是否可通行
    return maze[pos[0]][pos[1]]!=99


def find_path(maze,pos,end,last_step=(1,0)):
    mark(maze,pos)
    if last_step == (1,0):
        dirs = [(0,1),(1,0),(0,-1)]
    if last_step == (-1,0):
        dirs = [(0,1),(0,-1),(-1,0)]
    if last_step == (0,1):
        dirs = [(0,1),(1,0),(-1,0)]
    if last_step == (0,-1):
        dirs = [(1,0),(0,-1),(-1,0)]
    random.sample(dirs, len(dirs))
    dirs = sorted(dirs, key=lambda k: random.random())
    if pos==end:
#         print(pos,end=" ")  #已到达出口，输出这个位置。成功结束
        path.append(pos)
        return True
    for i in range(3):      #否则按四个方向顺序检查
        nextp = pos[0]+dirs[i][0],pos[1]+dirs[i][1]
        last_step = (dirs[i][0],dirs[i][1])
        #考虑下一个可能方向
        if passable(maze,nextp):        #不可行的相邻位置不管
            if find_path(maze,nextp,end,last_step):#如果从nextp可达出口，输出这个位置，成功结束
#                 print(pos,end=" ")
                path.append(pos)
                return True
    return False


def see_path(maze,path):     #使寻找到的路径可视化
    for i,p in enumerate(path):
        if i==0:
            maze[p[0]][p[1]] ="E"
        elif i==len(path)-1:
            maze[p[0]][p[1]]="S"
        else:
            maze[p[0]][p[1]] =3
    print("\n")
    for r in maze:
        for c in r:
            if c==3:
                print('\033[0;31m'+"*"+" "+'\033[0m',end="")
            elif c=="S" or c=="E":
                print('\033[0;34m'+c+" " + '\033[0m', end="")
            elif c==2:
                print('\033[0;32m'+"#"+" "+'\033[0m',end="")
            elif c==1:
                print('\033[0;;40m'+" "*2+'\033[0m',end="")
            else:
                print(" "*2,end="")
        print()

def map_value(maze,path):
    begin = 1 
    path = list(reversed(path))
    for i,p in enumerate(path):
        if maze[p[0]][p[1]] == 10:
            begin *= 2
        elif maze[p[0]][p[1]] == 20:
            begin *= 3
        else:
            begin += maze[p[0]][p[1]]
#         print("+ %s = %s" %(maze[p[0]][p[1]],begin))
    return begin
        
value  = []
allpath = []
for i in range(5000000):
    maze = [[99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99],\
            [99, 0, 1, 0, 2, 0, 8, 0, 5, 0, 3, 0, 1, 0, 1, 0, 99],\
            [99, -2, 99, 2, 99, 10, 99, 5, 99, -1, 99, 5, 99, 10, 99, -1, 99],\
            [99, 0, 2, 0, 7, 0, -1, 0, 2, 0, 2, 0, 3, 0, 5, 0, 99],\
            [99, 7, 99, 5, 99, 2, 99, -1, 99, 2, 99, 3, 99, 8, 99, 1, 99],\
            [99, 0, 3, 0, 2, 0, 6, 0, 4, 0, 2, 0, 1, 0, 8, 0, 99],\
            [99, 4, 99, 6, 99, -1, 99, 2, 99, -1, 99, 8, 99, -1, 99, 8, 99],\
            [99, 0, 2, 0, 5, 0, 3, 0, 4, 0, 1, 0, -1, 0, 3, 0, 99],\
            [99, 2, 99, 10, 99, 4, 99, 1, 99, 9, 99, 4, 99, 20, 99, 5, 99],\
            [99, 0, -2, 0, 3, 0, 4, 0, 7, 0, -1, 0, 1, 0, 3, 0, 99],\
            [99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99]]
    start=(9,1)
    end=(1,15)
    path=[]
    find_path(maze,start,end)
    allpath.append(path)
    value.append(map_value(maze_ori,path))

In [ ]:
maxval = np.max(value)
maxpos = np.argmax(value)

print(maxval)
print(list(reversed(allpath[maxpos])))
see_path(maze,allpath[maxpos])

In [292]:
maze_ori

[[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99],
 [99, 0, 1, 0, 2, 0, 8, 0, 5, 0, 3, 0, 1, 0, 1, 0, 99],
 [99, -2, 99, 2, 99, 10, 99, 5, 99, -1, 99, 5, 99, 10, 99, -1, 99],
 [99, 0, 2, 0, 7, 0, -1, 0, 2, 0, 2, 0, 3, 0, 5, 0, 99],
 [99, 7, 99, 5, 99, 2, 99, -1, 99, 2, 99, 3, 99, 8, 99, 1, 99],
 [99, 0, 3, 0, 2, 0, 6, 0, 4, 0, 2, 0, 1, 0, 8, 0, 99],
 [99, 4, 99, 6, 99, -1, 99, 2, 99, -1, 99, 8, 99, -1, 99, 8, 99],
 [99, 0, 2, 0, 5, 0, 3, 0, 4, 0, 1, 0, -1, 0, 3, 0, 99],
 [99, 2, 99, 10, 99, 4, 99, 1, 99, 9, 99, 4, 99, 20, 99, 5, 99],
 [99, 0, -2, 0, 3, 0, 4, 0, 7, 0, -1, 0, 1, 0, 3, 0, 99],
 [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99]]

In [341]:
def map_value(maze,path):
    begin = 1 
    path = list(reversed(path))
    for i,p in enumerate(path):
        if maze[p[0]][p[1]] == 10:
            begin *= 2
        elif maze[p[0]][p[1]] == 20:
            begin *= 3
        else:
            begin += maze[p[0]][p[1]]
        print("+ %s = %s" %(maze[p[0]][p[1]],begin))
    return begin

            
map_value(maze_ori,allpath[maxpos])
# see_path(maze,allpath[maxpos])

+ 0 = 1
+ -2 = -1
+ 0 = -1
+ 2 = 1
+ 0 = 1
+ 4 = 5
+ 0 = 5
+ 2 = 7
+ 0 = 7
+ 5 = 12
+ 0 = 12
+ -1 = 11
+ 0 = 11
+ 3 = 14
+ 0 = 14
+ 1 = 15
+ 0 = 15
+ 4 = 19
+ 0 = 19
+ 2 = 21
+ 0 = 21
+ 6 = 27
+ 0 = 27
+ 4 = 31
+ 0 = 31
+ -1 = 30
+ 0 = 30
+ 1 = 31
+ 0 = 31
+ -1 = 30
+ 0 = 30
+ 8 = 38
+ 0 = 38
+ 3 = 41
+ 0 = 41
+ 5 = 46
+ 0 = 46
+ 3 = 49
+ 0 = 49
+ 8 = 57
+ 0 = 57
+ -1 = 56
+ 0 = 56
+ 8 = 64
+ 0 = 64
+ 1 = 65
+ 0 = 65
+ 2 = 67
+ 0 = 67
+ 2 = 69
+ 0 = 69
+ 2 = 71
+ 0 = 71
+ 2 = 73
+ 0 = 73
+ -1 = 72
+ 0 = 72
+ 3 = 75
+ 0 = 75
+ 5 = 80
+ 0 = 80
+ 8 = 88
+ 0 = 88
+ 10 = 176
+ 0 = 176
+ 2 = 178
+ 0 = 178
+ -1 = 177
+ 0 = 177
+ 7 = 184
+ 0 = 184
+ 2 = 186
+ 0 = 186
+ 5 = 191
+ 0 = 191
+ 3 = 194
+ 0 = 194
+ 2 = 196
+ 0 = 196
+ 6 = 202
+ 0 = 202
+ 10 = 404
+ 0 = 404
+ 3 = 407
+ 0 = 407
+ 4 = 411
+ 0 = 411
+ 7 = 418
+ 0 = 418
+ 9 = 427
+ 0 = 427
+ -1 = 426
+ 0 = 426
+ 1 = 427
+ 0 = 427
+ 3 = 430
+ 0 = 430
+ 20 = 1290
+ 0 = 1290
+ 3 = 1293
+ 0 = 1293
+ 5 = 1298
+ 0 = 1298
+ 8 = 1306
+ 0 = 1306
+

2625

In [335]:
maze_ori = np.matrix("99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99;\
                 99, 0, 1, 0, 2, 0, 8, 0, 5, 0, 3, 0, 1, 0, 1, 0, 99;\
           99, -2, 99, 2, 99, 10, 99, 5, 99, -1, 99, 5, 99, 10, 99, -1, 99;\
           99, 0, 2, 0, 7, 0, -1, 0, 2, 0, 2, 0, 3, 0, 5, 0, 99;\
           99, 7, 99, 5, 99, 2, 99, -1, 99 2, 99, 3, 99, 8, 99, 1, 99;\
           99, 0, 3, 0, 2, 0, 6, 0, 4, 0, 2, 0, 1, 0, 8, 0, 99;\
           99, 4, 99, 6, 99, -1, 99, 2, 99, -1, 99, 8, 99, -1, 99, 8, 99;\
           99, 0, 2, 0, 5, 0, 3, 0, 4, 0, 1, 0, -1, 0, 3, 0, 99;\
           99, 2, 99, 10, 99, 4, 99, 1, 99, 9, 99, 4, 99, 20, 99, 5, 99;\
           99, 0, -2, 0, 3, 0, 4, 0, 7, 0, -1, 0, 1, 0, 3, 0, 99;\
           99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99")
print(maze_ori)

[[99 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99]
 [99  0  1  0  2  0  8  0  5  0  3  0  1  0  1  0 99]
 [99 -2 99  2 99 10 99  5 99 -1 99  5 99 10 99 -1 99]
 [99  0  2  0  7  0 -1  0  2  0  2  0  3  0  5  0 99]
 [99  7 99  5 99  2 99 -1 99  2 99  3 99  8 99  1 99]
 [99  0  3  0  2  0  6  0  4  0  2  0  1  0  8  0 99]
 [99  4 99  6 99 -1 99  2 99 -1 99  8 99 -1 99  8 99]
 [99  0  2  0  5  0  3  0  4  0  1  0 -1  0  3  0 99]
 [99  2 99 10 99  4 99  1 99  9 99  4 99 20 99  5 99]
 [99  0 -2  0  3  0  4  0  7  0 -1  0  1  0  3  0 99]
 [99 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99]]


In [8]:
def see_path(maze,path):     #使寻找到的路径可视化
    for i,p in enumerate(path):
        if i==0:
            maze[p[0]][p[1]] ="E"
        elif i==len(path)-1:
            maze[p[0]][p[1]]="S"
        else:
            maze[p[0]][p[1]] =3
        print("\n")
        for r in maze:
            for c in r:
                if c==3:
                    print('\033[0;31m'+"*"+" "+'\033[0m',end="")
                elif c=="S" or c=="E":
                    print('\033[0;34m'+c+" " + '\033[0m', end="")
                elif c==2:
                    print('\033[0;32m'+"#"+" "+'\033[0m',end="")
                elif c==1:
                    print('\033[0;;40m'+" "*2+'\033[0m',end="")
                else:
                    print('\033[0;;40m'+"# "*2+'\033[0m',end="")
#                     print(" "*2,end="")
            print()

(array([], dtype=int64),)